Notebook for working on IBM datascience capstone project

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
print("Hello Capstone Project Course!")

# Week 2 Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

Testing beautiful soup code
wikilink = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
html = bs4.BeautifulSoup(wikilink, "html.parser")
for text in html.select('table'):
    print(text.getText())

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

# Ignoring cells with a borough that is Not assigned

In [3]:
df.drop(df.loc[df['Borough']=="Not assigned"].index, inplace=True)

In [4]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [ ]:
#Mark the rows where neighborhood value is not assigned
df['NANeighborhood'] = np.where(df['Neighborhood'] == "Not Assigned", 'True', 'False')
df.head()

In [ ]:
# If value is not assigned, assign borough value to neighborhood
df['Neighborhood'] = np.where(df['NANeighborhood'] == True, df['Borough'],df['Neighborhood'])

In [ ]:
#Drop the extra column
df.drop(['NANeighborhood'], axis=1, inplace=True)
df.shape

In [ ]:
df.shape